# 前言  



📚提要：  

* 本期主要爬一爬当当网网站，看下过去的疫情四年（2020年至2023年），每年的图书畅销榜单TOP500都有哪些书？秉持尽可能多的获取网页上有的信息，满足小伙伴的不同需求，本次一共爬取12个字段数据，非常干货，搬起小板凳搞起来，如有任何问题的话欢迎私信交流；  

⏰温馨提示  

* 1）网站前端代码会随时变化，可能出现报错情况，爬取代码仅供参考；  
* 2）前面几个part环节代码仅作展示，可能存在部分变量未定义报错，若想要运行出结果，建议直接运行最终代码；  
* 3）由于部分代码过长，隐藏其中一部分，可点击运行Fork查看哦~；  

🏠个人专栏  
* [数分小白龙🐉](https://www.heywhale.com/home/column/65f93aebe948b593299975ec?shareby=5f0a61d72be199002c073935)  

📖其他爬虫项目推荐  👇👇  
* [【爬虫】中国电影市场近10年票房榜单数据](https://www.heywhale.com/mw/project/6627bd4aa67e459494d84abc)  
* [【爬虫】当当网2020-2023年畅销图书榜单](https://www.heywhale.com/mw/project/65f6fae03e36b30ae9e7e80e)  
* [【爬虫】链家网上海市在售新房楼盘数据](https://www.heywhale.com/mw/project/6628a7f95c42defa7a974592)  
* [【爬虫】华为Pura70微博热搜帖子数据（进阶版）](https://www.heywhale.com/mw/project/661e4800a3185f16a0177f2a)  
* [【爬虫】国内各品牌汽车销量排行榜单](https://www.heywhale.com/mw/project/6618f95e20e12fcbf348767a)  
* [【爬虫】小米雷神微博下网友评论数据](https://www.heywhale.com/mw/project/65f94992e948b593299bdfe9)  
* [【爬虫】微博<任意关键词>相关帖子数据（基础版）](https://www.heywhale.com/mw/project/65f1978693f67e9a14b3c836)  
* [【爬虫】交通局民航生产指标数据](https://www.heywhale.com/mw/project/65f1abc1617433dc66366aec)  
* [【爬虫】携程飞机航班数据](https://www.heywhale.com/mw/project/65f1a8b693f67e9a14b56535)  
* [【爬虫】安居客南京市在售房源数据](https://www.heywhale.com/mw/project/61bc07037e520c001798623c)  
* [【爬虫】房天下武汉市在售房源数据](https://www.heywhale.com/mw/project/624009ad40f3c800183801a6)  
* [【爬虫】Python零基础入门爬虫（含基础+五大案例）](https://www.heywhale.com/mw/project/60be1fa8acdcb3001707b133)  

# Part 1 目标网站

爬取的网站url = 'http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-2023-0-1-1'

![Image Name](https://cdn.kesci.com/upload/seb0rwwgk0.png?imageView2/0/w/960/h/960)

# Part 2 网站解析

本次主要爬取的字段有：年份、书籍排行、书名、评论数、推荐值、作者、出版日期、出版社、原价、折扣价、折扣比例、电子书价格共12个字段；


![Image Name](https://cdn.kesci.com/upload/seb0t0qceg.png?imageView2/0/w/960/h/960)  


通过检查网页源码发现，当当网是静态网页，只需在doc里找到html文件，依次搜索相关指标字段即可完成查询；

# Part 3 多页爬取  

本次有两处涉及循环多页爬取的地方：1：遍历每个年份，从2020年至2023年；2、遍历每年的图书榜单，共计25页；

## 年份遍历  

发现不同年份之间的url的唯一不同就是year后面的数字不一样；


http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-2020-0-1-1  
http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-2021-0-1-1  
http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-2022-0-1-1  
http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-2023-0-1-1

## 页面遍历  

遍历页面的话，是最后一个数字不一样；

http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-2023-0-1-1  
http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-2023-0-1-2  
http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-2023-0-1-3  
http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-year-2023-0-1-4

因此，可以写成http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-{year}-{page}这样的通用URL；

# Part 4 完整代码

## 导入模块

In [1]:
from lxml import etree
import requests
import random
import time
import csv
import pandas as pd 
import numpy as np
import re
import os

## 设置请求头

In [2]:
cookies = {
    'ddscreen': '2',
    'ddscreen': '2',
    'dest_area': 'country_id%3D9000%26province_id%3D111%26city_id%20%3D0%26district_id%3D0%26town_id%3D0',
    '__permanent_id': '20240423210658530124490989268736883',
    'MDD_channelId': '70000',
    'MDD_fromPlatform': '307',
    'ddscreen': '2',
    '__visit_id': '20240530154038979262380281306734049',
    '__out_refer': '',
    '__rpm': '...1717054859559%7C...1717054899777',
    '__trace_id': '20240530154142377181404279783243769',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ddscreen=2; ddscreen=2; dest_area=country_id%3D9000%26province_id%3D111%26city_id%20%3D0%26district_id%3D0%26town_id%3D0; __permanent_id=20240423210658530124490989268736883; MDD_channelId=70000; MDD_fromPlatform=307; ddscreen=2; __visit_id=20240530154038979262380281306734049; __out_refer=; __rpm=...1717054859559%7C...1717054899777; __trace_id=20240530154142377181404279783243769',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
}

## 爬取及解析函数

In [3]:
def SpiderData(url):
    response = requests.get(
        url,
        cookies=cookies,
        headers=headers,
        verify=False,
    )
    page_text = response.text
    return page_text

👇👇下面解析函数代码过长，仅展示一部分，详细代码可以点击右上角【Fork】按钮查看哈～


![Image Name](https://cdn.kesci.com/upload/seb0x6tirj.png?imageView2/0/w/960/h/960)  


In [4]:
def ParseData(page_text,key):
    tree = etree.HTML(page_text)
    lis = tree.xpath('.//ul[@class="bang_list clearfix bang_list_mode"]/li')
    rank, name, comments, recommends, author, publish_date, publish_house, original_price, discount_price, discount, ebook_price = [[] for i in range(11)]
    for li in lis:
        rank.append(''.join(li.xpath('.//div[@class="list_num red" or @class="list_num "]/text()')).replace('.',''))
        name.append(''.join(li.xpath('.//div[@class="name"]/a/text()')))
        comments.append(''.join(li.xpath('.//div[@class="star"]/a/text()')).split('条')[0])
        recommends.append(''.join(li.xpath('.//div[@class="star"]/span/text()')).split('推荐')[0])
        author.append(''.join(li.xpath('.//div[@class="publisher_info"][1]/a[1]/text()')))
        publish_date.append(''.join(li.xpath('.//div[@class="publisher_info"][2]/span/text()')))
        publish_house.append(''.join(li.xpath('.//div[@class="publisher_info"][2]/a/text()')))
        original_price.append(''.join(li.xpath('.//div[@class="price"]/p[1]/span[1]/text()')).replace("¥", ""))
        discount_price.append(''.join(li.xpath('.//span[@class="price_r"]/text()')).replace("¥", ""))
        discount.append(''.join(li.xpath('.//span[@class="price_s"]/text()')))
        ebook_price.append(''.join(li.xpath('./div[@class="price"]/p[@class="price_e"]/span[@class="price_n"]/text()')).replace("¥", ""))

    # print(original_price)
    #print(len(rank),len(name),len(comments),len(recommends),len(author),len(publish_date),len(publish_date),len(original_price),len(discount_price),len(discount))

    # 保存数据
    dic = {
        '排行榜类型':key,
        '排序':rank, 
        '书名':name, 
        '评论数':comments,
        '推荐值':recommends, 
        '作者':author, 
        '出版日期':publish_date, 
        '出版社':publish_date, 
        '原价':original_price,
        '折扣价':discount_price, 
        '折扣比例':discount,
        '电子书价格':ebook_price
    }
    df1 = pd.DataFrame(dic)
    return df1

## 主函数

In [5]:
if __name__ == "__main__":
    # 创建空数据框
    columns = ['排行榜类型', '排序', '书名', '评论数', '推荐值', '作者', '出版日期', '出版社', '原价','折扣价', '折扣比例', '电子书价格']
    df = pd.DataFrame(columns=columns)

    book_rank_type = {
        "2020年": "year-2020-0-1",
        "2021年": "year-2021-0-1",
        "2022年": "year-2022-0-1",
        "2023年": "year-2023-0-1"
    }

    # 循环爬取
    for key, value in book_rank_type.items():
        print(f'=====================开始爬{key}榜单数据===================')
        for page in range(25): # 排行榜共有25页数据
            print('*****************开始爬取第{}页数据*****************'.format(page+1))
            url = f'http://bang.dangdang.com/books/bestsellers/01.00.00.00.00.00-{value}-{page+1}'
            #print(url)
            # try:
            time.sleep(1)
            data = SpiderData(url)
            df1 = ParseData(data, key)
            k = len(df)
            df = pd.concat([df,df1], axis=0)
            df.drop_duplicates(subset=None, keep='first', inplace=True)
            print('*********第{}页数据爬取完成，爬取{}条数据，目前共爬取{}条数据**********'.format(page+1,len(df)-k,len(df)))
            # except:
            #     print('!!!!!!!!!第{}页数据爬取有误，需进行优化!!!!!!!!!'.format(page+1))
            #     break
        df = df.reset_index(drop=True)
        print(f'=================={key}榜单数据爬取完成，共有{len(df)}条数据===================')

=====================开始爬2020年榜单数据===================
*****************开始爬取第1页数据*****************
*********第1页数据爬取完成，爬取20条数据，目前共爬取20条数据**********
*****************开始爬取第2页数据*****************
*********第2页数据爬取完成，爬取20条数据，目前共爬取40条数据**********
*****************开始爬取第3页数据*****************
*********第3页数据爬取完成，爬取20条数据，目前共爬取60条数据**********
*****************开始爬取第4页数据*****************
*********第4页数据爬取完成，爬取20条数据，目前共爬取80条数据**********
*****************开始爬取第5页数据*****************
*********第5页数据爬取完成，爬取20条数据，目前共爬取100条数据**********
*****************开始爬取第6页数据*****************
*********第6页数据爬取完成，爬取20条数据，目前共爬取120条数据**********
*****************开始爬取第7页数据*****************
*********第7页数据爬取完成，爬取20条数据，目前共爬取140条数据**********
*****************开始爬取第8页数据*****************
*********第8页数据爬取完成，爬取20条数据，目前共爬取160条数据**********
*****************开始爬取第9页数据*****************
*********第9页数据爬取完成，爬取20条数据，目前共爬取180条数据**********
*****************开始爬取第10页数据*****************
*********第10页数据爬取完成，爬取20条数据，目前共爬取200条数据**********
*****************开始

*********第9页数据爬取完成，爬取20条数据，目前共爬取1680条数据**********
*****************开始爬取第10页数据*****************
*********第10页数据爬取完成，爬取20条数据，目前共爬取1700条数据**********
*****************开始爬取第11页数据*****************
*********第11页数据爬取完成，爬取20条数据，目前共爬取1720条数据**********
*****************开始爬取第12页数据*****************
*********第12页数据爬取完成，爬取20条数据，目前共爬取1740条数据**********
*****************开始爬取第13页数据*****************
*********第13页数据爬取完成，爬取20条数据，目前共爬取1760条数据**********
*****************开始爬取第14页数据*****************
*********第14页数据爬取完成，爬取20条数据，目前共爬取1780条数据**********
*****************开始爬取第15页数据*****************
*********第15页数据爬取完成，爬取20条数据，目前共爬取1800条数据**********
*****************开始爬取第16页数据*****************
*********第16页数据爬取完成，爬取20条数据，目前共爬取1820条数据**********
*****************开始爬取第17页数据*****************
*********第17页数据爬取完成，爬取20条数据，目前共爬取1840条数据**********
*****************开始爬取第18页数据*****************
*********第18页数据爬取完成，爬取20条数据，目前共爬取1860条数据**********
*****************开始爬取第19页数据*****************
*********第19页数据爬取完成，爬取20条数据，目前共爬取1880条数据*

## 数据展示

In [6]:
df.shape

(2000, 12)

In [7]:
df.head()

,书名,作者,出版日期,出版社,原价,折扣价,折扣比例,排序,排行榜类型,推荐值,电子书价格,评论数
0,你当像鸟飞往你的山（中文版销量超200万册，比尔・盖茨年度特别推,塔拉,2019-11-01,2019-11-01,34.80,59.00,5.9折,1,2020年,100%,24.99,1705641
1,人间失格（日本小说家太宰治代表作，一个对村上春树影响至深的绝,太宰治,2015-08-01,2015-08-01,9.80,25.00,3.9折,2,2020年,99.8%,4.99,2227301
2,乌合之众 : 大众心理研究（群体心理学创始人古斯塔夫・勒庞代表作,古斯塔夫・勒庞,2018-04-06,2018-04-06,10.10,26.00,3.9折,3,2020年,100%,,752375
3,神奇校车・图画书版（全12册）,乔安娜柯尔,2018-05-10,2018-05-10,165.00,198.00,8.3折,4,2020年,99.9%,,1784300
4,作家榜名著：月亮与六便士（159万+读者高分推荐版！好评率高达99,毛姆,2017-01-10,2017-01-10,39.80,39.80,10.0折,5,2020年,100%,3.99,1590551


## 数据保存

In [8]:
df.to_excel('当当网近4年畅销图书榜单数据.xlsx',header=True,index=False)

*好了，本项目到现在为止基本就算完成了，如果后续有更好的想法可以留言或者私信哈，由于做的时间比较匆忙，不完善的地方敬请谅解；*  

*最后，欢迎大家点赞、收藏、评论、Fork，动动小手指一起来学习呀！关注我，后续继续不断更新爬虫项目～*